### Gromacs MD轨迹处理

In [ ]:
# conda install -c conda-forge mdanalysis
import torch 
import MDAnalysis as mda

In [3]:
# load the topological file
u = mda.Universe('./protein_conf.gro', ['./protein_snap_every200ps_00to02us.xtc'])
print(u)
print(len(u.trajectory))

<Universe with 9364 atoms>
10000


In [6]:
u.trajectory[0].__dict__

{'frame': 0,
 '_n_atoms': 9364,
 'data': {'time': 200.00003051757812, 'step': 80000, 'dt': 200.00003051757812},
 '_has_positions': True,
 '_has_velocities': False,
 '_has_forces': False,
 '_has_dimensions': False,
 '_pos': array([[36.54    , 46.43    , 34.940002],
        [36.600002, 45.480003, 35.25    ],
        [35.58    , 46.47    , 34.620003],
        ...,
        [69.350006, 37.280003, 63.440002],
        [68.9     , 37.58    , 64.600006],
        [69.44    , 36.14    , 62.890007]], dtype=float32),
 '_unitcell': array([99.892105, 99.892105, 99.892105, 90.      , 90.      , 90.      ],
       dtype=float32),
 'aux': {}}

In [7]:
# 将velocities的属性设为True
u.trajectory.ts.has_velocities = True
u.trajectory[0].__dict__

In [43]:
atom_class = {}
atoms = []

for i in u.atoms.types:
    if i not in atom_class.keys():
        atoms.append(len(atom_class) + 1)
        atom_class[i] = len(atom_class) + 1
    else:
        atoms.append(atom_class[i])

atoms = torch.tensor(atoms)
print(atoms.shape, len(atom_class))

9364 5


In [42]:
vel = []
for i in range(len(u.trajectory) - 1):
    vel.append(torch.tensor(u.trajectory[i + 1]._pos)  - torch.tensor(u.trajectory[i]._pos))

In [45]:
vel.append(torch.tensor(u.trajectory[-1]._pos))

In [47]:
vel = torch.stack(vel)
vel.shape

torch.Size([10000, 9364, 3])

In [49]:
pos = []
for i in range(len(u.trajectory)):
    pos.append(torch.tensor(u.trajectory[i]._pos))

pos = torch.stack(pos)
pos.shape

torch.Size([10000, 9364, 3])

In [50]:
torch.save([atoms, pos, vel], 'trajectory_10K.pt')

In [62]:
import numpy as np

In [63]:
np.savez('trajetory_10K', atoms=atoms.numpy(), pos=pos.numpy, vel=vel.numpy())

In [61]:
atoms.numpy(), pos.numpy, vel.numpy()

array([1, 2, 2, ..., 3, 4, 4])